In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('JenniferPriceData.csv')
data=data.iloc[:,1:]

In [3]:
data.head(5)

,Name,Date,Price,DailyChange,WeekChange,MarketCap,DailyVolume,lag1_DailyChange,lag2_DailyChange,lag3_DailyChange,lag1_WeekChange,lag1_DailyRank,lag2_DailyRank,lag3_DailyRank,lag1_WeekRank,lagN_Price,response
0,Ethereum,2017-01-04,11.25,15.40,47.31,985511573.0,41051196.0,16.33,3.03,1.78,35.83,0.060606,0.454545,0.393939,0.121212,10.25,-8.89
1,Ethereum,2017-01-05,10.25,-9.17,22.95,898495324.0,41557420.0,15.40,16.33,3.03,47.31,0.303030,0.060606,0.454545,0.181818,10.25,0.00
2,Ethereum,2017-01-06,10.25,-1.08,26.05,898841639.0,29471794.0,-9.17,15.40,16.33,22.95,0.545455,0.303030,0.060606,0.212121,9.87,-3.71
3,Ethereum,2017-01-07,9.87,-3.92,21.76,865194304.0,23153574.0,-1.08,-9.17,15.40,26.05,0.303030,0.545455,0.303030,0.121212,10.29,4.26
4,Ethereum,2017-01-08,10.29,4.41,26.16,902575822.0,16676626.0,-3.92,-1.08,-9.17,21.76,0.878788,0.303030,0.545455,0.181818,10.33,0.39


## Start from data preprocessing

In [4]:
# three matrix: return , price and sort, also the market capitalization matrix

In [5]:
name,length=np.unique(data['Name'].values,return_counts=True)
max_length = max(length)

In [6]:
%%time
price =[]
return_arr = []
mar_cap = []
for i in range(len(name)):
    df=data[data['Name']==name[i]]
    p=df['Price'].values
    r=(df['Price'].diff()/df['Price']).values
    cap=df['MarketCap'].values
    emp = np.empty((max_length-length[i],))
    emp[:] = np.NaN
    price.append(np.concatenate((emp,p)))
    return_arr.append(np.concatenate((emp,r)))
    mar_cap.append(np.concatenate((emp,cap)))

CPU times: user 19.2 s, sys: 16.7 ms, total: 19.2 s
Wall time: 19.2 s


In [7]:
price_df=pd.DataFrame(price)
price_df.reindex(name)
return_df=pd.DataFrame(return_arr)
return_df.reindex(name)

,0,1,2,3,4,5,6,7,8,9,...,1508,1509,1510,1511,1512,1513,1514,1515,1516,1517
0x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0xcert,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1SG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1irstcoin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
indaHash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nDEX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pEOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
qiibee,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
replace_0_return=np.nan_to_num(return_arr,0)
sort=np.argsort(replace_0_return,axis=0)

In [9]:
### market return for each day(weighted return based on market portfolio)

In [96]:
replace_0_cap=np.nan_to_num(mar_cap,0)
sort_cap=np.argsort(replace_0_cap,axis=0)

In [12]:
mar_return=np.sum(replace_0_return*replace_0_cap*(1/np.sum(replace_0_cap,axis=0)),axis=0)

In [13]:
mar_return

array([ 0.        , -0.08722966, -0.0282749 , ..., -0.03333066,
        0.08033156, -0.02791203])

## select the top 5 and bottom 5

### simple select top x and bottom x and discover whether they acheive the momentum/mean reversion

In [14]:
# let's start with 200 as most of the token did not exsit at the early stage

In [20]:
sort

array([[  0,  19, 194, ..., 361,  11, 158],
       [429, 281, 180, ..., 205,  65, 648],
       [430, 180, 489, ..., 255, 319, 365],
       ...,
       [221, 223,  51, ..., 113, 611, 319],
       [203, 206, 281, ..., 187,  82, 361],
       [650,  51,  19, ..., 440, 439,  11]])

In [46]:
def simple_ratio_momentum(top_num,lag,start,market):
    count = 0
    for i in range(np.shape(return_arr)[0]):
        for j in range(start-lag,np.shape(return_arr)[1]-lag):
            if sort[i][j]<top_num:
                if return_arr[i][j+lag]>=(mar_return[j+lag] if market else 0):
                    count+=1
            
    return count/(np.shape(return_arr)[1]-start)/top_num

In [51]:
simple_ratio_momentum(10,1,1000,True),simple_ratio_momentum(10,1,1000,False)

(0.4557915057915058, 0.5231660231660232)

In [102]:
sort_rev = -1*(sort-650)

In [94]:
def simple_ratio_mean_reversion(bottom_num,lag,start,market):
    count = 0
    for i in range(np.shape(return_arr)[0]):
        for j in range(start-lag,np.shape(return_arr)[1]-lag):
             if sort_rev[i][j]<bottom_num:
                if return_arr[i][j+lag]>=(mar_return[j+lag] if market else 0):
                    count+=1
            
    return count/(np.shape(return_arr)[1]-start)/bottom_num

In [95]:
simple_ratio_mean_reversion(10,1,1000,True),simple_ratio_mean_reversion(10,1,1000,False)

(0.4555984555984556, 0.5301158301158301)

In [99]:
from itertools import product
list(product([1,2,3],[5,10,20]))

[(1, 5), (1, 10), (1, 20), (2, 5), (2, 10), (2, 20), (3, 5), (3, 10), (3, 20)]

In [131]:
for i,j in list(product([1,2,3],[5,10,20])):
    print(i,j)
    print(simple_ratio_momentum(j,i,800,True),simple_ratio_momentum(j,i,800,False))
    print(simple_ratio_mean_reversion(j,i,800,True),simple_ratio_mean_reversion(j,i,800,False))

1 5
0.4337047353760446 0.49164345403899723
0.4473537604456824 0.5022284122562674
1 10
0.4350974930362117 0.4910863509749303
0.44164345403899724 0.49665738161559886
1 20
0.4403899721448468 0.4956128133704736
0.4435933147632312 0.49616991643454045
2 5
0.45403899721448465 0.49832869080779946
0.4403899721448468 0.49136490250696385
2 10
0.44944289693593314 0.4948467966573816
0.4419220055710307 0.49526462395543175
2 20
0.4454038997214485 0.4935933147632312
0.4433147632311978 0.49867688022284123
3 5
0.45125348189415043 0.5033426183844012
0.45348189415041784 0.5091922005571031
3 10
0.45 0.5023676880222842
0.45208913649025073 0.5052924791086351
3 20
0.44568245125348194 0.5028551532033426
0.44742339832869077 0.5023676880222842


# conclusion: 
1.using only mean reversion or momentum is no better than random guess, compared to the market portfolio, however, the return of market portfolio is highly influenced by the token with high marketcap. Also that prediction accuracy is probably not enough to cover the transaction cost.  
2.Inspire me to introduct two baselines, EWP and market portfolio

In [ ]:
### baselines:

In [60]:
mar_return+1

array([1.        , 0.91277034, 0.9717251 , ..., 0.96666934, 1.08033156,
       0.97208797])

In [81]:
mar_cum_return=np.cumprod(mar_return[800:]+1)

In [82]:
ewp_return=np.sum(replace_0_return/651,axis=0)
ewp_cum_return=np.cumprod(ewp_return[800:]+1)

In [147]:
ewp_return 

array([ 0.        , -0.00266941, -0.00269656, ..., -0.04979369,
        0.03597428, -0.02446379])

In [84]:
## both ewp and market portfolio is not a good baslines.

In [189]:
def fetch_data_momentum(bottom_num,lag,start):

    #sort_cap_rev= -1*(sort_cap-650)
    arr = []
    for j in range(start-lag,np.shape(return_arr)[1]-lag):
        arr_i=[]
        for i in range(np.shape(return_arr)[0]):
             if sort[i][j]<bottom_num:
                    arr_i.append(return_arr[i][j+lag])
        arr.append(arr_i)
                                     
    return arr

In [190]:
data_momentum=np.nan_to_num(fetch_data_momentum(5,1,800),0)
ewp_momentum=np.sum(data_momentum/5,axis=1)

In [139]:
def fetch_data_mean_reversion(bottom_num,lag,start):

    #sort_cap_rev= -1*(sort_cap-650)
    arr = []
    for j in range(start-lag,np.shape(return_arr)[1]-lag):
        arr_i=[]
        for i in range(np.shape(return_arr)[0]):
             if sort_rev[i][j]<bottom_num:
                    arr_i.append(return_arr[i][j+lag])
        arr.append(arr_i)
                                     
    return arr

In [150]:
data_mean_reversion=np.nan_to_num(fetch_data_mean_reversion(5,1,800),0)
ewp_mean_reversion=np.sum(data_mean_reversion/5,axis=1)
np.cumprod(ewp_mean_reversion+1).shape

(718,)

In [148]:
### simple selection

In [198]:
def fetch_data_constraint(bottom_num,cap,lag,start):
    #sort_cap_rev= -1*(sort_cap-650)
    arr = []
    for j in range(start-lag,np.shape(return_arr)[1]-lag):
        arr_i=[]
        for i in range(np.shape(return_arr)[0]):
             if sort_cap[i][j]<cap and sort_rev[i][j]<bottom_num:
                    arr_i.append(return_arr[i][j+lag])
        return_sum=np.sum(arr_i)
        arr.append(return_sum)
    
    return arr

In [202]:
data_mean_reversion=np.nan_to_num(fetch_data_constraint(10,100,1,800),0)
np.cumprod(data_mean_reversion+1)

array([ 9.43141593e-01,  9.29122574e-01,  9.93289939e-01,  9.93289939e-01,
        9.81513821e-01,  9.81513821e-01,  8.81795038e-01,  8.95714455e-01,
        8.95416776e-01,  8.95416776e-01,  8.40855664e-01,  8.40855664e-01,
        6.37771458e-01,  6.58410725e-01,  7.19005392e-01,  7.19005392e-01,
        7.54240232e-01,  7.25980091e-01,  7.25980091e-01,  7.25980091e-01,
        7.52140364e-01,  7.52140364e-01,  6.87948757e-01,  6.87948757e-01,
        6.87948757e-01,  5.64829229e-01,  5.72829903e-01,  7.60316975e-01,
        5.22431281e-01,  5.22431281e-01,  1.86817624e-01,  1.94464071e-01,
        2.08446177e-01,  2.08446177e-01,  2.08446177e-01,  2.03155040e-01,
        2.03155040e-01,  2.03155040e-01,  1.99444446e-01,  1.99444446e-01,
        1.99444446e-01,  1.99444446e-01,  2.22021438e-01,  1.90683279e-01,
        1.53610559e-01,  1.53610559e-01,  1.53610559e-01,  1.53610559e-01,
        1.44261480e-01,  1.44261480e-01,  1.48885590e-01,  6.92394584e-02,
        6.92394584e-02,  

In [ ]:
t